In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from numpy import mean
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import re
import string
from string import punctuation
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm
import googletrans
from googletrans import *
from langdetect import detect

import warnings
warnings.filterwarnings('ignore')

## Sentiment Analysis

In [3]:
sia = SentimentIntensityAnalyzer()

In [ ]:
# SENTIMENT ANALYSIS DF1

scores = []
for tweet in tqdm(df1['TWEET'], desc='Sentiment Analysis'):
    score = sia.polarity_scores(tweet)
    scores.append(score) 

scores_df = pd.DataFrame(scores)

df1 = pd.concat([df1, scores_df], axis=1)

In [ ]:
# Assign Sentiment label

df1['SENTIMENT'] = df1['compound'].apply(lambda x: 'Positive' if x >= 0.05 else ('Negative' if x <= -0.05 else 'Neutral'))

In [ ]:
# # TEXT PROCESSING DF2 MAKES NO DIFFERENCE

# df2 = df1.copy()
# tqdm.pandas()
# def get_text_processing(text):
#     stpword = stopwords.words('english')
#     no_punctuation = [char for char in text if char not in string.punctuation]
#     no_punctuation = ''.join(no_punctuation)
#     return ' '.join([word for word in no_punctuation.split() if
#                     word.lower() not in stpword])

# tqdm.pandas(desc="Text Processing")
# df2['N_T'] = df2['TWEET'].progress_apply(get_text_processing)
# #df2['N_T'] = df2['TWEET'].apply(get_text_processing)

# # SENTIMENT ANALYSIS DF2

# scores2 = []
# tqdm.pandas(desc="Sentiment Analysis")
# for tweet in tqdm(df2['N_T']):
#     score2 = sia.polarity_scores(tweet)
#     scores2.append(score2) 

# scores_df2 = pd.DataFrame(scores)

# df2 = pd.concat([df2, scores_df2], axis=1)

# df2['SENTIMENT2'] = df2['compound'].apply(lambda x: 'Positive' if x >= 0.05 else ('Negative' if x <= -0.05 else 'Neutral'))

In [ ]:
# Translate neutral tweets

# def translate_tweets(df_n):
#     translator = googletrans.Translator()
#     translated_tweets = []
    
#     for tweet in tqdm(df_n['TWEET']):
#         try:
#             if len(tweet.strip()) > 0 and len(tweet.split()) > 2:
#                 if detect(tweet) != 'en':
#                     tweet = str(tweet)
#                     translated_tweet = translator.translate(tweet, src='auto', dest='en').text
#                 else:
#                     translated_tweet = tweet
#             else:
#                 translated_tweet = tweet
                
#         except (ValueError, AttributeError, Exception) as e:
#             translated_tweet = np.nan
        
#         translated_tweets.append(translated_tweet)
    
#     df_n['TWEET'] = translated_tweets
    
#     return df_n


In [ ]:
#df_n = translate_tweets(df_n)

In [ ]:
# Create df by day

df1['DATE'] = pd.to_datetime(df1['DATE'])
df1['DATE_DAY'] = df1['DATE'].dt.date
df1_day = df1.groupby('DATE_DAY').agg({'DAY_OF_WEEK': 'first', 'TWEET': 'count', 'compound': 'mean'}).reset_index()
df1_day['SENTIMENT'] = df1_day['compound'].apply(lambda x: 'Positive' if x >= 0.05 else ('Negative' if x <= -0.05 else 'Neutral'))
df1_day.rename(columns={'DATE_DAY' : 'DATE', 'TWEET': 'TWEETS'}, inplace=True)


## Sentiment Analysis with ROBERTA

In [ ]:
from transformers import TFRobertaForSequenceClassification, RobertaTokenizer
import tensorflow as tf

In [ ]:
from tqdm.notebook import tqdm  # Importar tqdm_notebook en lugar de tqdm

# Cargar el modelo preentrenado y el tokenizador
model_name = "roberta-base"  # Puedes cambiar esto según el modelo que quieras utilizar
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = TFRobertaForSequenceClassification.from_pretrained(model_name)

# Supongamos que 'text' es la columna en tu DataFrame que contiene los textos que quieres analizar
texts = df1['TWEET'].tolist()

# Tokenizar los textos y convertirlos en vectores de entrada
encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='tf')

# Realizar la inferencia sobre los datos
predictions = []

# Agregar una barra de progreso
with tqdm(total=len(encoded_texts['input_ids'])) as pbar:
    for i in range(len(encoded_texts['input_ids'])):
        # Hacer la predicción para un solo texto
        pred = model.predict({k: v[i:i+1] for k, v in encoded_texts.items()})
        predictions.append(pred)
        pbar.update(1)  # Actualizar la barra de progreso

# Obtener las etiquetas de los sentimientos
sentiments = tf.argmax(predictions.logits, axis=1)

# Agregar los resultados de sentimientos al DataFrame original
df1['sentiment'] = sentiments.numpy()
